# **Importando Bibliotecas**

In [1]:
!pip install pdfplumber -q
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.6 MB/s eta 0:00:00


In [2]:
import pdfplumber
import os
import pandas as pd
import xlsxwriter
from pprint import pprint

# **Funcoes**

In [3]:
def dadosPagina(pdf,pagina):
    page = pdf.pages[pagina]
    head = page.extract_text_simple()
    head = head.split('\n')
    cabecalho = []
    table = page.extract_table()
    return head,table

def cabecalho(pdf,pagina):
    page = pdf.pages[pagina]
    head = page.extract_text_simple()
    head = head.split('\n')
    return head


# **Agendamento**

In [4]:
pdf = pdfplumber.open('/content/Relatório.pdf')
qtdPages = len(pdf.pages)

In [5]:
#Pegando o numero de pagina por profissional

profPaginas = {}

for pagina in range(0,qtdPages):
    head,table = dadosPagina(pdf,pagina)
    paginas = []
    profissional = []
    paginas = []

    if pagina < qtdPages-1:
        head1, table1 = dadosPagina(pdf,pagina+1)
        try:
            head2, table2 = dadosPagina(pdf,pagina+2)
        except IndexError:
            pass
        try:
            head3, table3 = dadosPagina(pdf,pagina+3)
        except IndexError:
            pass

    if 'Profissional' in  head[5]:
        profissional.append(head[5][14:])
        paginas.append(pagina)
        if 'Profissional' not in head1[5]:
            paginas.append(pagina+1)
            if 'Profissional' not in head2[5]:
                if pagina+2 != qtdPages:
                    paginas.append(pagina+2)
                if'Profissional' not in head3[5]:
                    if pagina+3 != qtdPages:
                        paginas.append(pagina+3)
        profPaginas[profissional[0]] = paginas

In [6]:
#Pegando todos os pacientes que possuem consulta por medico:

profPacientes = {}
for prof, paginas in profPaginas.items():
    linhas = []
    if len(paginas) > 1:
        for pagina in paginas:
            head,table = dadosPagina(pdf,pagina)
            if pagina == paginas[0]:
                for linha in table[1:]:
                    linhas.append(linha)
            elif pagina != paginas[0]:
                for linha in table[0:]:
                    linhas.append(linha)
        profPacientes[prof] = linhas
    if len(paginas) <= 1:
        head,table = dadosPagina(pdf,paginas[0])
        for linha in table[1:]:
            linhas.append(linha)
        profPacientes[prof] = linhas

#Pegando a especialidade de cada médico:
profEspecialidade = {}
for pagina in range(0,qtdPages):
    head,table = dadosPagina(pdf,pagina)
    if "Profissional" in head[5]:
        profEspecialidade[head[5][14:]] = head[7][15:]

#Convertendo a coluna de prontuarios para o formato INT:
for prof, linha in profPacientes.items():
    for x in range(0,len(linha)):
        prontuario = profPacientes[prof][x][3]
        try:
            profPacientes[prof][x][3] = int(prontuario)
        except:
            pass

#Ordenando a coluna prontuarios em ordem crescente
for prof, linha in profPacientes.items():
    temp = {}
    sortedTemp = {}
    contador = 1
    for x in range(0,len(linha)):
        prontuario = profPacientes[prof][x][3]
        if type(prontuario) == type('teste'):
            prontuario = contador
            contador = contador + 1
        temp[prontuario] = linha[x]
    for x in sorted(temp):
        sortedTemp[x] = temp[x]
    profPacientes[prof] = sortedTemp.values()

In [25]:
workbook = xlsxwriter.Workbook("Agendamento.xlsx")
cell_format1 = workbook.add_format()
cell_format2 = workbook.add_format()
cell_format3 = workbook.add_format()
tamanho_col = {1:173.68,2:85.41,3:86.55,4:175,5:99.04,6:175,7:80.125,8:133}
pagina = 0
colunas = ["Horário", "Paciente", "Data Nasc.", "Prontuário", "Exames",
            "Telefone", "Observação", "CNS", "Seq. Atendimento",]

for prof, pacientes in profPacientes.items():
    contador = 10
    contador2 = 6
    contador3 = 0
    worksheet = workbook.add_worksheet(f"Pagina {pagina+1}")
    head = []

    if len(profPacientes[prof]) > 13:
        head = cabecalho(pdf,profPaginas[prof][0])
        #Inserindo o cabecalho* da pagina
        for linha in range(0,8):
            cell_format1.set_bold()
            cell_format1.set_font_size(12)
            worksheet.write(linha,0,head[linha],cell_format1)

        for linha in range(0,14):
            cell_format2.set_align('vcenter')
            cell_format2.set_align('justify')
            cell_format2.set_border(1)
            value = list(profPacientes[prof])
            for coluna in [0,1,2,3,4,5,6,7,8]:
                worksheet.write(contador,coluna,value[linha][coluna],cell_format2)

                if coluna == 5:
                    if value[linha][5].count('\n') == 0 and value[linha][7].count('\n') == 0:
                        pass
                    elif value[linha][5].count('\n') > value[linha][7].count('\n'):
                        if value[linha][5].count('\n') == 1:
                            worksheet.set_row(contador,25)
                        elif value[linha][5].count('\n') == 2:
                            worksheet.set_row(contador,35)
                        elif value[linha][5].count('\n') == 3:
                            worksheet.set_row(contador,48)
                    elif value[linha][7].count('\n') > value[linha][5].count('\n'):
                        worksheet.set_row(contador,26)
                    elif value[linha][7].count('\n') == 1 and value[linha][5].count('\n') == 1:
                        worksheet.set_row(contador,26)
            contador += 1
        worksheet.set_landscape()
        worksheet.print_area('A1:I28')
        worksheet.set_print_scale(90)
        worksheet.set_margins(left=0.20, right=0.10, top=0.15, bottom=0.15)

        for coluna in [0,1,2,3,4,5,6,7,8]:
            cell_format3.set_bg_color('#B4B4B4')
            cell_format3.set_bold()
            cell_format3.set_font_size(12)
            worksheet.write(9,coluna,colunas[coluna],cell_format3)

        #Alterando o tamanho das colunas:
        for key, value in tamanho_col.items():
            worksheet.set_column_pixels(key,key,value)

        #Inserindo o final da planilha.
        worksheet.write(26,0,head[-2],cell_format1)
        worksheet.write(27,0,head[-1],cell_format1)

        pagina += 1
        worksheet = workbook.add_worksheet(f"Pagina {pagina+1}")
        head = cabecalho(pdf,profPaginas[prof][1])

        for linha in range(0,4):
            worksheet.write(linha,0,head[linha],cell_format1)

        for linha in range(14,len(pacientes)):
            value = list(profPacientes[prof])
            for coluna in [0,1,2,3,4,5,6,7,8]:
                worksheet.write(contador2,coluna,value[linha][coluna],cell_format2)
                if coluna == 5:
                    if value[linha][5].count('\n') == 0 and value[linha][7].count('\n') == 0:
                        pass
                    elif value[linha][5].count('\n') > value[linha][7].count('\n'):
                        if value[linha][5].count('\n') == 1:
                            worksheet.set_row(contador2,25)
                        elif value[linha][5].count('\n') == 2:
                            worksheet.set_row(contador2,35)
                        elif value[linha][5].count('\n') == 3:
                            worksheet.set_row(contador2,48)
                    elif value[linha][7].count('\n') > value[linha][5].count('\n'):
                        worksheet.set_row(contador2,26)
                    elif value[linha][7].count('\n') == 1 and value[linha][5].count('\n') == 1:
                        worksheet.set_row(contador2,26)
            contador2 += 1

        #Alterando as configuracoes de pagina da planilha
        worksheet.set_landscape()
        worksheet.print_area('A1:I28')
        worksheet.set_print_scale(90)
        worksheet.set_margins(left=0.20, right=0.10, top=0.15, bottom=0.15)


        for coluna in [0,1,2,3,4,5,6,7,8]:
            cell_format3.set_bg_color('#B4B4B4')
            cell_format3.set_bold()
            cell_format3.set_font_size(12)
            worksheet.write(5,coluna,colunas[coluna],cell_format3)
    #Alterando o tamanho das colunas:
        for key, value in tamanho_col.items():
            worksheet.set_column_pixels(key,key,value)

    #Inserindo o final da planilha.
        worksheet.write(26,0,head[-2],cell_format1)
        worksheet.write(27,0,head[-1],cell_format1)

        pagina += 1

    elif len(profPacientes[prof]) <= 13:
        head = cabecalho(pdf,profPaginas[prof][0])
        #Inserindo o cabecalho* da pagina
        for linha in range(0,8):
            cell_format1.set_bold()
            cell_format1.set_font_size(12)
            worksheet.write(linha,0,head[linha],cell_format1)

        for linha in range(0,len(pacientes)):
            cell_format2.set_align('vcenter')
            cell_format2.set_align('justify')
            cell_format2.set_border(1)
            value = list(profPacientes[prof])
            for coluna in [0,1,2,3,4,5,6,7,8]:
                worksheet.write(contador,coluna,value[linha][coluna],cell_format2)
                if coluna == 5:
                    if value[linha][5].count('\n') == 0 and value[linha][7].count('\n') == 0:
                        pass
                    elif value[linha][5].count('\n') > value[linha][7].count('\n'):
                        if value[linha][5].count('\n') == 1:
                            worksheet.set_row(contador,25)
                        elif value[linha][5].count('\n') == 2:
                            worksheet.set_row(contador,35)
                        elif value[linha][5].count('\n') == 3:
                            worksheet.set_row(contador,48)
                    elif value[linha][7].count('\n') > value[linha][5].count('\n'):
                        worksheet.set_row(contador,26)
                    elif value[linha][7].count('\n') == 1 and value[linha][5].count('\n') == 1:
                        worksheet.set_row(contador,26)
            contador += 1
        worksheet.set_landscape()
        worksheet.print_area('A1:I28')
        worksheet.set_print_scale(90)
        worksheet.set_margins(left=0.20, right=0.10, top=0.15, bottom=0.15)


        for coluna in [0,1,2,3,4,5,6,7,8]:
            cell_format3.set_bg_color('#B4B4B4')
            cell_format3.set_bold()
            cell_format3.set_font_size(12)
            worksheet.write(9,coluna,colunas[coluna],cell_format3)

        #Alterando o tamanho das colunas:
        for key, value in tamanho_col.items():
            worksheet.set_column_pixels(key,key,value)

        #Inserindo o final da planilha.
        worksheet.write(26,0,head[-2],cell_format1)
        worksheet.write(27,0,head[-1],cell_format1)

        pagina += 1

workbook.close()